In [ ]:
# conda install -c conda-forge langdetect

In [ ]:
# pip install venmo-api

In [ ]:
# conda install -c conda-forge spacy

In [ ]:
# python -m spacy download en_core_web_sm
# ^^ note this was done in terminal and did not work within the notebook

In [ ]:
# pip install emoji

In [11]:
import pandas as pd
import pickle
from langdetect import detect
import datetime
import re
import emoji
from textblob import TextBlob

In [12]:
transactions = pickle.load(open("Data/transactions.pickle","rb"))

In [13]:
user_list = pickle.load(open("Data/user_list.pickle","rb"))

In [6]:
# Transactions is a list of transaction objects (venmo API)\
# User_list is a list of user objects (venmo API)

In [14]:
len(transactions)

286464

Convert the transactions into a dataframe

step 1 - pull out all the relevant attributes from the transaction objects
Note that the actor and target attributes themselves are each of objects of the user class. For now, just need to pull the id attribute from there to match up ids.

In [15]:
story_id = [x.id for x in transactions]
payment_id = [x.payment_id for x in transactions]
date_completed = [x.date_completed for x in transactions]
date_created = [x.date_created for x in transactions]
date_updated = [x.date_updated for x in transactions]
payment_type = [x.payment_type for x in transactions]
amount = [x.amount for x in transactions]
audience = [x.audience for x in transactions]
status = [x.status for x in transactions]
note = [x.note for x in transactions]
device_used = [x.device_used for x in transactions]
actor_id = [x.actor.id for x in transactions]

cannot use a list comprehension for target_id because some transactions do not have a target (e.g. cancelation. see transactions 26176)

In [16]:
counter = 0
target_id = []
target_issue = []
for x in transactions:
    try:
        target_id.append(x.target.id)
    except:
        target_issue.append([counter, x.id])
    counter += 1

# combine all series into dataframe

In [17]:
frames = [story_id, payment_id, date_completed, date_created, date_updated, payment_type, amount, audience, status, note, device_used, actor_id, target_id]
column_names = {0:'story_id', 1:'payment_id', 2:'date_completed', 3:'date_created', 4:'date_updated', 5:'payment_type', 6:'amount', 7:'audience', 8:'status', 9:'note', 10:'device_used', 11:'actor_id', 12:'target_id'}

In [18]:
df = pd.DataFrame(frames)
df = df.T
df = df.rename(columns = column_names)

In [19]:
def convert_date(Series):
    '''
    Need to use a special function because some entries will say 'None', because not all transactions are completed.
    So, using normal datetime conversions wont work.
    '''
    converted = []
    counter = 0
    for x in Series:
        try:
            converted.append(datetime.datetime.fromtimestamp(x))
        except:
            converted.append(None)
        counter += 1
    return pd.to_datetime(converted)

In [20]:
df.date_completed = convert_date(df.date_completed)
df.date_created = convert_date(df.date_created)
df.date_updated = convert_date(df.date_updated)

In [21]:
with open('Data/clean_df.pickle', 'wb') as f:
    pickle.dump(df, f)

Create a dictionary to lookup any users

In [22]:
users = dict()
for user in user_list:
    users[user.id] = user

# preprocess the text data

In [23]:
#convert everything to lowercase
#remove punctuations
#correct minor mispelling with multiple 

In [24]:
# Apply a first round of text cleaning techniques
def clean_text_round1(text):
    '''Make text lowercase, remove punctuation, remove words containing numbers, and
    for letters that show up 3 or more times consecutively, convert to just one
    '''
    text = text.lower()
    text = re.sub('[\.\,\!\?\:\;\-\=\#\$\&\(\)\+\/\/\@]', '', text) #note that '_' is preserved to be handled later
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"([a-z])\1\1+",r"\1\1",text)
    return text

In [31]:
notes_r1 = [clean_text_round1(x) for x in df.note]

## Let's talk Emojis

Many challenges with emojis. See three sample issues below

Problem 1: Emoji characters are typed without spaces:
'ugh taxes😬' <- Emoji is tacked onto another word. Need to add a space to separate them \
'🥪🥪🥪' <- Oftentime emojis will be repeated for emphasis. Need to add a space so that it is not treated as one word\

    **Solution**: This can be solved by adding a space prior to every emoji


Problem 2: Sometimes emojis are used like letters.
'🍞🍗🍞' <-  This example uses emojis to create a chicken sandwich, by using :bread::poultry_leg::bread:

    **Solution**: One solution is to leave the emojis as is, and this will then be treated as one word. This would only work if emojis are left in as characters, and not demojized (translated).

Problem: 
'🍿🍿🍿' <- popcorn is meaningful, and i want this entry to be considered the same as 'popcorn'. To make this interpretable, need to get emojis into words

    **Solution**: The emoji library can demojize into :popcorn::popcorn::popcorn:


**Options**:

**A. Spaced_notes:** Keep emojis, with addition of spaces between emojis  
**B. Defined:** add the demojis, with spaces between emojis and demojis  
**C. Singled_defined:** add demojis, without spaces between emoji and demoji (but between one set of emoji-demoji and the next)  
**D. Demoji_notes:** translate the emojis into demojis, removing the emojis. This may be better for traditional natural language processing techniques that are not accustomed to one character words (which an emoji is)  
**E. Single_demoji_notes:** same as demoji_notes, but keeping the emoji-demoji together without spaces  
**F. Keep emojis, no spaces** <- listing this for completeness, but will not use given that it isn't going to help much. 

**Champion Model:** After trying a few things out, developed a custom processing technique

### Spaced_Notes
- Keep emojis, with addition of spaces between emojis.
- This prevents emojis that are listed together from being treated as one "word"

In [86]:
def spaced_emoji(text):
    emoji_list = emoji.emoji_lis(text)
    new_text = str()
    if emoji_list == []:
        return text
    for num in range(len(emoji_list)-1,-1,-1):
        i = emoji_list[num]['location']
        x = emoji_list[num]['emoji']
        new_text = text[:i] + ' ' + text[i:i+1] + ' ' + new_text[i+1:]
        new_text = re.sub('[\.\,\!\?\:\;\-\=\_\'\%\(\)\¯\\\¯]', ' ', new_text)
    return new_text

In [87]:
notes_spaced_emoji = [spaced_emoji(x) for x in notes_r1]

In [88]:
with open('Data/spaced_notes.pickle', 'wb') as f:
    pickle.dump(notes_spaced_emoji, f)

### Defined_Notes
Add emoji translations to the text, so that I can associate emojis with certain words. That way those notes with emojis only or those with text only can start to be grouped together

This may train the model to associate words with emojis. which may help with topic modeling, and getting popcorn and 🍿 into the same topics without needing an additional pre-processing step during production. Also, would be helpful for text generation, if we want to create an emoji suggestor based on various words/topics

In [89]:
def defined_emoji(text):
    emoji_list = emoji.emoji_lis(text)
    new_text = str()
    if emoji_list == []:
        return text
    for num in range(len(emoji_list)-1,-1,-1):
        i = emoji_list[num]['location']
        x = emoji_list[num]['emoji']
        new_text = text[:i] + ' ' + text[i:i+1] + ' ' +emoji.demojize(x) + ' ' + new_text[i+1:]
        new_text = re.sub('[\.\,\!\?\:\;\-\=\_\'\%\(\)\¯\\\¯]', ' ', new_text)
    return new_text

In [90]:
# example of defined_emoji
emoji_text = '🍿🍿🍿'

defined_emoji(emoji_text)

' 🍿  popcorn   🍿  popcorn   🍿  popcorn  '

In [91]:
defined_notes = [defined_emoji(x) for x in notes_r1]

In [92]:
with open('Data/defined_notes.pickle', 'wb') as f:
    pickle.dump(defined_notes, f)

## Single_Defined
same as defined_notes, however this does not add spaces between the emojis. Without spaces, then the emoji and its definition will be considered one word. This may help to consider emojis separately from those who spell it out(hamburger vs the emoji hamburger). Could be useful when emojis take on a slightly different meaning/usage than the demojized word

In [26]:
def single_defined_emoji(text):
    emoji_list = emoji.emoji_lis(text)
    new_text = str()
    if emoji_list == []:
        return text
    for num in range(len(emoji_list)-1,-1,-1):
        i = emoji_list[num]['location']
        x = emoji_list[num]['emoji']
        new_text = text[:i] + ' ' + text[i:i+1] + ' ' +emoji.demojize(x) + ' ' + new_text[i+1:]
        new_text = re.sub('[\.\,\!\?\:\;\-\=\'\%\(\)\¯\\\¯]', '', new_text)
    return new_text

In [27]:
single_defined_notes = [single_defined_emoji(x) for x in notes_r1]

In [28]:
with open('Data/single_defined_notes.pickle', 'wb') as f:
    pickle.dump(single_defined_notes, f)

## Demoji_notes

### demoji + remove foreign characters
 - demojize the note, remove punctuation
 - considered to remove non-latin characters, but since it is small part of data set, instead just set a minimum number in the vectorizer and that will get rid of them.

In [32]:
#basic demojizer with underscore removal
def demojize_clean(text):
    text = emoji.demojize(text)
    text = re.sub('[\.\,\!\?\:\;\-\=\_\'\%\(\)\¯\\\¯]', ' ', text)
    return text
# reference: https://stackoverflow.com/questions/23680976/python-removing-non-latin-characters

In [33]:
demoji_notes = [demojize_clean(x) for x in notes_r1]

In [34]:
with open('Data/demoji_notes.pickle', 'wb') as f:
    pickle.dump(demoji_notes, f)

In [32]:
#basic demojizer with underscore removal
def single_demojize_clean(text):
    text = emoji.demojize(text)
    text = re.sub('[\.\,\!\?\:\;\-\=\_\'\%\(\)\¯\\\¯]', '', text)
    return text
# reference: https://stackoverflow.com/questions/23680976/python-removing-non-latin-characters

In [33]:
single_demoji_notes = [single_demojize_clean(x) for x in notes_r1]

In [34]:
with open('Data/single_demoji_notes.pickle', 'wb') as f:
    pickle.dump(single_demoji_notes, f)

## CHAMPION PROCESSING
- Converts all emojis into words (demojis)
- Custom changes to the words/demojis so that word embeddings from Google-news-300 can be used.
- Note that in the Word_Vector_matrix file, additional special processing will be done where **new** words such as "lyft" or "venmo" will be have embeddings from replacement words ("cab" or "paypal") because the Google-news-300 dataset is from 2012, prior to the newer technologies being adopted.

In [41]:
#keep demojis with the underscore
def champion_demojize_clean(text):
    text = emoji.demojize(text)
    
    #custom definition for interprebility against Google training set
    text = text.replace('hot_dog', 'hotdog')
    text = text.replace('hot_beverage', 'coffee') #this 
    text = text.replace('sport_utility_vehicle', 'SUV') 
    text = text.replace('shallow_pan_of_food', 'paella')  #https://emojipedia.org/shallow-pan-of-food/
    text = text.replace('cup_of_water', 'soda')
    text = text.replace('money_with_wings', 'money')
    text = text.replace('wrapped_gift', 'gift')

    #remove punctuations, especially key to get rid of semicolons and underscores
    text = re.sub('[\.\,\!\?\:\;\-\=\_\'\%\(\)\¯\\\¯]', ' ', text)
    
    #add back underscore for connected words/emojis that Google data can understand
    text = text.replace('french fries', 'french_fries')
    text = text.replace('bento box', 'bento_box')
    text = text.replace('light bulb', 'light_bulb')
    text = text.replace('beer mugs', 'beer_mugs')
    text = text.replace('clinking glasses', 'clinking_glasses')
    text = text.replace('steaming bowl', 'steaming_bowl')
    text = text.replace('fried shrimp', 'fried_shrimp')
    text = text.replace('santa claus', 'santa_claus')
    text = text.replace('palm tree', 'palm_tree')
    text = text.replace('cherry blossom', 'cherry_blossom')
    
    #custom definition for interprebility (done after the fact because of the underscore)
    text = text.replace('poultry leg', 'chicken_drumstick')
    
    
    #custom stop words
    text = text.replace('oncoming', '') #oncoming_automobile
    text = text.replace('selector', '') #airplane_selector red_heart_selector      

    #other notes:
    # dog_face and pig_face will generate the terms: dog, pig, face. leaving this as is to get the face
    # ok_hand will become ok, hand. Then ok will be removed in later vectorizer option for only 3 letter words.
    # other strange emoji definitions: telephone_receiver, zany_hearts, rocket_popsicle. last of which is a custom venmo made for festivals
    
    return text
# reference: https://stackoverflow.com/questions/23680976/python-removing-non-latin-characters

In [42]:
champion_demoji_notes = [champion_demojize_clean(x) for x in notes_r1]

In [43]:
with open('Data/champion_demoji_notes.pickle', 'wb') as f:
    pickle.dump(champion_demoji_notes, f)

In [46]:
champion_demoji_notes.index('bnb')

38083

## Remove foreign languages -DEPRECATED IDEA

Note that I attempted to remove foreign languages using the langdetect library. 

However, this method was ultimately not useful because many documents (transaction notes) showed up as other languages such as de (because of uber)

still, had some learnings:
1. Emoji only notes are the most common type of notes by far (interpreted as 'None' language)
    - therefore it is very important to account for emojis properly
2. Every transaction must have something in the note

In [12]:
#note that detect algorithm is slow to run. Took about one hour to run this on the 280k dataset
lang = []
counter = 0
for x in df.note:
    try:
        lang.append(detect(x))
    except:
        lang.append('None')
    if counter %1000 == 0:
        print(counter)
    counter += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [40]:
lang_series = pd.DataFrame({'language':lang})

In [52]:
note_frame = pd.concat([df.note,lang_series],axis=1)

In [245]:
# there are no empty notes
note_frame[note_frame.note=='']

Empty DataFrame
Columns: [note, language]
Index: []

In [102]:
note_frame.language.value_counts()

None     92169
en       76680
de       17316
so        7168
tl        6662
af        6346
fr        6261
it        5659
cy        5175
id        5087
ro        4764
nl        4762
da        4135
ca        3836
no        3832
tr        3710
sv        3608
es        3414
sw        3087
vi        2840
pt        2827
pl        2726
et        2656
fi        2595
lt        1535
sq        1504
sl        1411
sk         991
hr         964
lv         862
hu         770
cs         688
ko         152
ja         141
zh-cn       69
el          13
ru          11
ar           8
he           7
uk           7
fa           4
th           3
bg           3
ne           3
zh-tw        1
ur           1
mk           1
Name: language, dtype: int64

In [118]:
note_frame.groupby('language')['note'].unique()

language
None     [💇, 👿, 🎳, 🍵, ⛽, ✊🏿, 💃, 🌮🌮, 🥑, 🙏🏼, 🔥🍗, 🐔, 🎫, 🍿,...
af       [Hawt sliders, Jackie hotel 🏨, water, water!, ...
ar       [فووووووووود, صصسييدزف يحييك كان. جلسة سنشري س...
bg                        [Борщ, Жранина🐓🐠🍷🌷, Битва за 4$]
ca       [Tequilla, Kids a square, formal (x2), Electri...
cs       [Pickles, Photos, Love you 💜💎💍📿🌴🌞💙, Mcdicks🍟, ...
cy       [Alvin’s dynasty $, dynasty, Mod, Gold, Legall...
da       [Tbell, Evening fun, Dave, Jeremy, 🏐 refund, H...
de       [FF, sUpReMe, :uber:, SF, Lunch!, Dnr3, Dinna,...
el       [Ρ, ΥΟΓΓ, 🚗ΔΓ, 🍻  QTπ, Γοω τΗε βοατ, ( ^ω^ ), ...
en       [Food, Chack, Toyota. The whole company, She b...
es       [EScaPe, fugacious 💩, Electric, Ague para/por ...
et       [Massage, Test, Shakes 🥤, Shakes! 🧉, Shakes🥤, ...
fa             [دجاج, ᕦ(ò_óˇ)ᕤ٩( ᐛ )و(￣^￣)ゞ🍣🍣, ها, مارينا]
fi       [Tattoo, ☕️ and jimmy johns, Kennison💗, Kennis...
fr       [Jesse!!, Lo, Lauren Daigle tickets!, Lrc, Ken...
he       [גוי 🤮, מעגל, ח, יין 🎊, ‏סתומה, שמואל 

need to remove the documents with following languages 'ar', 'bg', 'el','fa','he','ja','ko''mk','ne','ru','th','uk','ur','zh-cn','zh-tw'

It is a little unfortunate to remove some tranasactions because those of a different language can still have emojis that provide information, or use text to show emotion such as ¯\_(ツ)_/¯ is labeled as japanese

In [121]:
sum(note_frame.language.isin(['ar', 'bg', 'el','fa','he','ja','ko','mk','ne','ru','th','uk','ur','zh-cn','zh-tw']))

424

In [122]:
df = df[note_frame.language.isin(['ar', 'bg', 'el','fa','he','ja','ko','mk','ne','ru','th','uk','ur','zh-cn','zh-tw'])==False]

## Autocorrect - DEPRECATED

Attempted Textblob for autocorrect but it is not good with double letters. Decided against autocorrect (specifically with Textblob and more generally with other libraries), because phones will typically have autocorrect built-in, so when a user enters a new transaction there should be few misspellings unless it is done on purpose (adding extra letters which i have addressed already in clean_round1.)

In [354]:
b = TextBlob('chikin')
print(b.correct())
c = TextBlob('awwesome')
print(c.correct())

chin
awwesome


unfortunately, 'chikin', spelled incorrectly on purpose will just need to be that way

## Create a Database - NOT USED
the code below is a work-in-progress. To be implemented in the future. Leaving this in here in case someone else is interested in using Google Cloud SQL Proxy

In [ ]:
# conda install psycopg2
# OR

# pip install psycopg2-binary <- to avoid building from source which is what would be needed without the -binary

https://cloud.google.com/sql/docs/postgres/connect-compute-engine#debianubuntu_2
Before starting: Make sure that the SQL instance is set up in the same project as the JupyterNotebook instance

- Follow instructions there, using Cloud SQL Proxy
- When it asks to open up a terminal, this can be done in JupyterLab (File>New Launcher>Terminal)
- step 6 - use the code for Debian/Ubuntu 
    - Can check this is the right OS within the terminal using "cat /etc/os-release" 
    - which i found here: https://www.cyberciti.biz/faq/how-to-check-os-version-in-linux-command-line/ because my guess was that the jupyter/python instance set up through GCP would be Linux.
- step 7 - I used the Linux-64-bit code to install the proxy
- step 8 - IN NEW TERMINAL - ran code in steps b and c
    - note that for step c need to pull the instance name for the SQL instance
        - because i set up the SQL instance in same project as JupyterNotebook, i dont need to deal with credentials
- step 9 - INSTEAD OF RUNNING the psql in terminal, ran the connection through psycopg2 below.
    - note that need to install psycopg2. this can be done through conda or pip...
        - for pip use: "pip install psycopg2-binary"
            - must use binary or it wont work here
        - for conda use: "conda install psycopg2"

In [ ]:
import psycopg2 as pg
import pandas as pd
import psycopg2.extras as extras

In [ ]:

#use following in a Terminal
# ./cloud_sql_proxy -dir=/cloudsql -instances=regal-subject-286004:us-west2:venmo-database
# or
# ./cloud_sql_proxy -dir=/cloudsql &

In [ ]:
conn = pg.connect(host='/cloudsql/regal-subject-286004:us-west2:venmo-database',dbname="venmo", user="postgres", password="secret")

In [ ]:
notes[20000:20010]

In [ ]:
print(user_list[0])

get everything into a SQL database

In [ ]:
#helper function to be used below
def execute_values(conn, df, table,columns):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df[columns]))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, pg.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [ ]:
df.columns

In [ ]:
execute_values(conn,df,'transactions',df.columns)

In [ ]:
df.iloc[1]

In [ ]:
type(df.iloc[:,1])